In [1]:
import pandas as pd
import math
import sqlite3 as sql
import csv

In [2]:
# Connect to database using sqlite3
con = sql.connect("chinook.db")

In [8]:
# a) Write an SQL query to gather information about the artists, their albums, tracks, genres, and playlists. Import this information into a pandas data frame.

query = '''
    SELECT r.ArtistId, r.Name, l.AlbumId, l.Title, t.TrackId, t.Name, g.GenreId, g.Name, p.PlaylistId, p.Name
    FROM Track t
    JOIN Album l USING(AlbumId)
    JOIN Artist r USING(ArtistId)
    JOIN Genre g USING(GenreId)
    JOIN PlaylistTrack USING(TrackId)
    JOIN Playlist p USING(PlaylistId);
    '''

music_df = pd.read_sql_query(query, con)

print(music_df)

      ArtistId           Name  AlbumId  \
0            8     Audioslave      271   
1            8     Audioslave      271   
2            8     Audioslave      271   
3            8     Audioslave      271   
4            8     Audioslave      271   
...        ...            ...      ...   
8710       114  Ozzy Osbourne      171   
8711       114  Ozzy Osbourne      171   
8712       114  Ozzy Osbourne      172   
8713       179      Scorpions      257   
8714        68    Miles Davis       48   

                                                  Title  TrackId  \
0                                           Revelations     3402   
1                                           Revelations     3389   
2                                           Revelations     3390   
3                                           Revelations     3391   
4                                           Revelations     3392   
...                                                 ...      ...   
8710               

In [10]:
# 10 points) Select all the artists that have more than one album for the analysis below
grouped_df = music_df.groupby(['ArtistId']).filter(lambda x: x['AlbumId'].nunique() > 1)

print(grouped_df.head(20))

    ArtistId        Name  AlbumId         Title  TrackId  \
0          8  Audioslave      271   Revelations     3402   
1          8  Audioslave      271   Revelations     3389   
2          8  Audioslave      271   Revelations     3390   
3          8  Audioslave      271   Revelations     3391   
4          8  Audioslave      271   Revelations     3392   
5          8  Audioslave      271   Revelations     3393   
6          8  Audioslave      271   Revelations     3394   
7          8  Audioslave      271   Revelations     3395   
8          8  Audioslave      271   Revelations     3396   
9          8  Audioslave      271   Revelations     3397   
10         8  Audioslave      271   Revelations     3398   
11         8  Audioslave      271   Revelations     3399   
12         8  Audioslave      271   Revelations     3400   
13         8  Audioslave      271   Revelations     3401   
40         8  Audioslave       11  Out Of Exile       99   
41         8  Audioslave       11  Out O

In [12]:
print(grouped_df.groupby(['ArtistId']))